## PyTorch Tutorial 10 - Dataset Transforms

### PyTorch comes with various built-in transforms

'''
Transforms can be applied to PIL images, tensors, ndarrays, or custom data
during creation of the DataSet

complete list of built-in transforms: 
https://pytorch.org/docs/stable/torchvision/transforms.html

### On Images
---------
CenterCrop, Grayscale, Pad, RandomAffine
RandomCrop, RandomHorizontalFlip, RandomRotation
Resize, Scale

### On Tensors
----------
LinearTransformation, Normalize, RandomErasing

### Conversion
----------
ToPILImage: from tensor or ndrarray
ToTensor : from numpy.ndarray or PILImage

### Generic
-------
Use Lambda 

### Custom
------
Write own class

### Compose multiple Transforms
---------------------------
composed = transforms.Compose([Rescale(256),
                               RandomCrop(224)])
'''

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader

import numpy as np
import math

In [17]:
class WineDataset(Dataset):

    def __init__(self, transform=None):
        
        # super(WineDataset, self).__init__() # calling this crashes kernel

        xy = np.loadtxt("./data/wine/wine.csv", delimiter=",", dtype=np.float32, skiprows = 1) # skip first header
        self.x = xy[:,1:] # converted to tensor
        self.y = xy[:,[0]]
        self.n_samples = xy.shape[0]

        self.transform = transform

    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)

        return sample # returns tuple of data

    def __len__(self):
        return self.n_samples # return len samples
    

In [18]:
# Customer transform class

class ToTensor:

    def __call__(self, sample): # callable class
        inputs, targets = sample
        
        return torch.from_numpy(inputs), torch.from_numpy(targets)

In [19]:
dataset = WineDataset(transform=ToTensor()) # create WineDataset object and pass the ToTensor

In [20]:
print(dataset[0]) # checking

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]), tensor([1.]))


### Applying multiple transformations

In [21]:

# Our second transformation
class MulTransform:

    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, target = sample
        inputs *= self.factor
        return inputs, target

In [22]:
# Here we use torchvision to compose both the transformations

composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])

In [23]:
dataset = WineDataset(transform=composed)

In [24]:
dataset[0] # check and observe that features and target are tensors as well as multiplied by 2

(tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
         6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
         2.1300e+03]),
 tensor([1.]))